In [1]:
import pandas as pd 
df = pd.read_csv("data\gemstone.csv")

In [2]:
df = df.drop("id" , axis = 1)

In [3]:
df.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [4]:
# Independent and dependent features 
X = df.drop("price", axis = 1)
Y = df[["price"]]

In [5]:
X.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77


In [6]:
Y.head()

,price
0,13619
1,13387
2,2772
3,666
4,14453


In [7]:
# Define which columns should be ordinal-encoded and which should be scaled
categorical_columns = X.select_dtypes(include="O").columns
numerical_columns = X.select_dtypes(exclude="O").columns

In [8]:
categorical_columns

Index(['cut', 'color', 'clarity'], dtype='object')

In [9]:
numerical_columns

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [10]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [11]:
from sklearn.impute import SimpleImputer # Handling missing values 
from sklearn.preprocessing import StandardScaler # Handling feature scalling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding --> Categorical features 

# Pipelines 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer # --> To combine two pipelines 

In [12]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())

    ]

)

# Categorigal Pipeline
cat_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
    ('scaler',StandardScaler())
    ]

)

preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_columns),
('cat_pipeline',cat_pipeline,categorical_columns)
])


In [18]:
preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')),
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Fair',
                                                                              'Good',
                                                                              'Very '
                                                                              'Good',
                                                                              'Premium',
                                                                              'Ideal'],
                                                                             ['D',
                                                                              'E',
                                                                              'F',
                                                                              'G',
                                                                              'H',
                                                                              'I',
                                                                              'J'],
                                                                             ['I1',
                                                                              'SI2',
                                                                              'SI1',
                                                                              'VS2',
                                                                              'VS1',
                                                                              'VVS2',
                                                                              'VVS1',
                                                                              'IF']])),
                                                 ('scaler', StandardScaler())]),
                                 Index(['cut', 'color', 'clarity'], dtype='object'))])

In [13]:
from sklearn.model_selection import train_test_split 

X_train , X_test , y_train , y_test = train_test_split(X,Y , test_size = 0.33 , random_state=42)

In [14]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [15]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,2.626061,-2.888129,0.400868,2.233112,2.216066,1.856561,-0.130933,1.525655,-1.314696
1,-0.845291,0.164716,0.922458,-0.915966,-0.908068,-0.890852,-0.130933,-0.937159,-0.648656
2,-0.845291,-1.500472,1.965640,-0.843987,-0.899013,-0.963153,-0.130933,-0.321455,-0.648656
3,-0.694363,-0.667878,-0.642314,-0.637048,-0.636405,-0.673951,0.874463,-0.937159,-1.314696
4,1.548002,-0.482857,1.444049,1.477333,1.455407,1.393839,-0.130933,1.525655,0.683424


In [16]:
X_test.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.629679,0.257226,-0.120723,-0.601058,-0.582072,-0.572731,0.874463,-1.552862,-0.648656
1,2.604500,-2.148045,-0.120723,2.125144,2.197955,1.957782,-1.136330,0.294248,-1.314696
2,-1.125587,-1.222941,0.922458,-1.374832,-1.415174,-1.469255,-0.130933,-0.937159,2.015504
3,-1.017781,-0.575368,0.922458,-1.158895,-1.161621,-1.194514,-0.130933,1.525655,2.015504
4,0.858043,0.627268,-0.642314,0.946489,0.984523,1.003417,0.874463,0.909952,-0.648656


In [21]:
y_test.head()

,price
14868,1355
165613,14691
96727,844
145593,707
118689,5797


In [24]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,2.626061,-2.888129,0.400868,2.233112,2.216066,1.856561,-0.130933,1.525655,-1.314696
1,-0.845291,0.164716,0.922458,-0.915966,-0.908068,-0.890852,-0.130933,-0.937159,-0.648656
2,-0.845291,-1.500472,1.965640,-0.843987,-0.899013,-0.963153,-0.130933,-0.321455,-0.648656
3,-0.694363,-0.667878,-0.642314,-0.637048,-0.636405,-0.673951,0.874463,-0.937159,-1.314696
4,1.548002,-0.482857,1.444049,1.477333,1.455407,1.393839,-0.130933,1.525655,0.683424


In [25]:
y_train.head()

,price
47047,16075
37559,904
96522,1016
151589,1173
98360,11655


In [26]:
regression = LinearRegression()
regression.fit(X_train , y_train)

LinearRegression()

In [27]:
regression.coef_

array([[ 6432.59272318,  -133.11853452,   -70.36485019, -1713.18964719,
         -490.48291102,   -68.02812257,    68.36709467,  -464.25812278,
          651.94096231]])

In [28]:
regression.intercept_

array([3979.27372333])

In [33]:
from sklearn.linear_model import LinearRegression , Lasso , Ridge , ElasticNet 
from sklearn.metrics import r2_score , mean_absolute_error , mean_squared_error 

In [34]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [35]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    
    # Here the models getting trained 
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')



LinearRegression
Model Training Performance
RMSE: 1014.274933056864
MAE: 674.7352796098307
R2 score 93.63893549824441


Lasso
Model Training Performance
RMSE: 1014.3366158273099
MAE: 675.8986621286324
R2 score 93.63816178295377


Ridge
Model Training Performance
RMSE: 1014.2792052203266
MAE: 674.7687088427467
R2 score 93.63888191205453


Elasticnet
Model Training Performance
RMSE: 1534.0746306294475
MAE: 1061.3169023914195
R2 score 85.4483784776376


